In [ ]:
# !conda install -c conda-forge python-dotenv -y

import numpy as np
import pandas as pd
from deap import base, creator, tools
from functools import reduce
from dotenv import load_dotenv
import finlab
import os
from finlab import data
from finlab.backtest import sim
from finlab.dataframe import FinlabDataFrame
from finlab.portfolio import Portfolio


# 載入.env檔案中的環境變數

load_dotenv()
# 這裡要替換成自己的FinLab VIP token
token = os.environ.get("FINLAB_TOKEN")
finlab.login(token)



## sharp ratio 簡單講，就是「報酬 / 風險」！
finlab文章:[https://www.finlab.tw/python%E6%96%B0%E6%89%8B%E6%95%99%E5%AD%B8%EF%BC%9A%E9%A2%A8%E9%9A%AA%E8%88%87%E5%A0%B1%E9%85%AC/]  
以這著比喻，可以想像，sharp ratio 越高，代表獲利大於風險，  
而sharp ratio 越低，代表風險大於獲利，那就會有點危險了！  
所以找一個sharp ratio 越高的指數，就等於找出了「獲利大且風險相對小」的指數喔！

---

### 如何定義獲利？  
獲利可以用每天平均的漲跌來代表，也就是今天漲1％，明天跌1％，平均獲利就是0％，  
接下來我們就用python來計算每天平均獲利吧  
 
---

### 計算sharpe ratio  
sharpe = profit / risk * (252 ** 0.5)  
可以看到上述程式，我們額外乘了一個「252 ** 0.5」  
因為我們希望算年化 annual sharpe ratio，  
其中的252是一年大約的交易天數，  
而「**」是「次方」的意思。  
為什麼要乘這個常數？最主要是因為大家幫自己的歷史回測計算sharpe ratio時候，都有乘上這個數字，要乘了才有辦法跟別人比較XD

In [ ]:
# 選擇特定市場 
with data.universe('TSE_OTC'):
    close = data.get('price:收盤價')
    print(close['2330'].iloc[-1])
    adj_close = data.get('etl:adj_close') # 還原收盤價
    print(adj_close['2330'].iloc[-1])

    
# ---- 報酬 -----
pct_change = adj_close.pct_change() # 漲跌幅
profit = pct_change.mean() # 平均漲跌幅 2007年~now
profit_rank=profit.sort_values(ascending=False) # 排名 ,進行由小到大的排序,acending=False就會變由大變小
print("profit",profit_rank)

# ---- 風險 -----
risk = pct_change.std() #風險通常會用標準差（standard deviation）
print("risk",risk)

# ---- sharpe -----
sharpe = profit / risk * (252 ** 0.5)
sharpe.sort_values()


## 加上移動窗格,改良上述的時間軸,因為上面是抓2007到現在,市場變化已經太大  
我們希望用時間移動窗格，每日都計算252天以前的sharpe值  

profit = pct_change.rolling(252).mean()  
risk = pct_change.rolling(252).std()  
sharpe = profit / risk * (252 ** 0.5)  

幾乎長的一模一樣對吧？唯一不一樣的是rolling(252)這個功能，
這是移動窗格252天的意思。
額外要注意的是，之前的寫法中，sharpe是一個series，index為指數名稱，而在現在的寫法中，sharpe變成了一個dataframe（table），其index代表日期，而columns代表每檔指數，其中的數值是 252 天的 sharpe ratio，神奇吧！

In [ ]:
pct_change = adj_close.pct_change() # 漲跌幅
profit = pct_change.rolling(252).mean() # 近一年平均漲跌幅
risk = pct_change.rolling(252).std() # 近一年風險標準差
sharpe = profit / risk * (252 ** 0.5) # 近一年sharpe ratio
# print("sharpe:",sharpe)

# 1. 提取2025-03-14那一天的數據
target_date = '2025-03-14'
target_row = sharpe.loc[target_date]

# 2. 將該行轉換為Series並刪除NaN值
target_row_clean = target_row.dropna()

# 3. 對該行數據進行排序（降序）
sorted_row = target_row_clean.sort_values(ascending=False)

print("sorted_row:",sorted_row)

# 4. 添加排名
sorted_row_with_rank = pd.DataFrame({
    'symbol': sorted_row.index,
    'value': sorted_row.values,
    'rank': range(1, len(sorted_row) + 1)
})

# 5. 輸出結果
print("\n2025-03-14那一天的數據排序結果（降序）：")
print(sorted_row_with_rank)


# 6. 如果想要查看前幾名和後幾名
print("\n前五名：")
print(sorted_row_with_rank.head(5))

print("\n後五名：")
print(sorted_row_with_rank.tail(5))


## 總和成函式
1. 設定平均漲跌幅,風險標準差跟sharpe值的「 天數 」
2. 輸出一個dataFrame可以用在finlab的因子


In [ ]:
def calcSharpe(pct_days,risk_days,date):
    pct_change = adj_close.pct_change() # 還原漲跌幅
    profit = pct_change.rolling(pct_days).mean() # 近X天平均漲跌幅
    risk = pct_change.rolling(risk_days).std() # 近X天風險標準差
    sharpe = profit / risk * (pct_days ** 0.5) # 近X天sharpe ratio
    #=== 輸出
    target_date = date
    target_row = sharpe.loc[target_date]
    target_row_clean = target_row.dropna()
    sorted_row = target_row_clean.sort_values(ascending=False)
    # 4. 添加排名
    sorted_row_with_rank = pd.DataFrame({
        'symbol': sorted_row.index,
        'value': sorted_row.values,
        'rank': range(1, len(sorted_row) + 1)
    })

    # 5. 輸出結果
    print(f"\n{date}那一天的數據排序結果（降序）：")
    print(sorted_row_with_rank.head(10))
    return sharpe

# 漲跌幅抓少天,風險標準差抓多天,可以抓到剛起漲飆股
# pct_days=5,risk_days=20
# pct_days=3,risk_days=20
sharpe_rank = calcSharpe(3,20,'2025-02-10')
print(sharpe_rank)

/var/folders/p6/syyz959s6tgfxwjns19hts0w0000gn/T/ipykernel_2668/3190209383.py:2: FutureWarning: The default fill_method='pad' in FinlabDataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  pct_change = adj_close.pct_change() # 還原漲跌幅



2025-02-10那一天的數據排序結果（降序）：
  symbol     value  rank
0   4946  4.357688     1
1   5530  3.915040     2
2   5548  3.724687     3
3   4722  3.543060     4
4   6482  3.524232     5
5   6113  3.483892     6
6   6654  3.447850     7
7   1815  3.420567     8
8   6846  3.347928     9
9   6919  3.344701    10
symbol          1101      1102      1103      1104  1107      1108      1109  \
date                                                                           
2007-04-23       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-24       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-25       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-26       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
2007-04-27       NaN       NaN       NaN       NaN   NaN       NaN       NaN   
...              ...       ...       ...       ...   ...       ...       ...   
2025-03-10 -0.410132  0.417938  0.003110 -0.737469   NaN  